**Preprocessing**

In [1]:
#all the needed Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

In [2]:
df = pd.read_excel("ServiceNowQuestions_clean.xlsx")

In [3]:
df.drop('Tags', inplace=True, axis=1)
df.drop('index', inplace=True, axis=1)
df.drop('Title', inplace=True, axis=1)
df.drop('level_0', inplace=True, axis=1)

In [4]:
df = df.astype({"Question": str})

In [5]:
# min_ = 100
# question_len = []
# print(question_len)
# for index, row in df.iterrows():
#          question_len.append(len(row["Question"]))


# fig = plt.figure(figsize =(10, 7))
 
# # Creating plot
# plt.boxplot(question_len)
 
# # show plot
# plt.show()
# print(np.max(question_len))
# print(np.min(question_len))
#question_len_df.describe()

In [6]:
#remove outliers
index_list = []
for index, row in df.iterrows():
        if((len(row["Question"]) < 15) | (len(row["Question"])> 861)):
            df.drop(index, inplace=True)


# question_len = []
# print(question_len)
# for index, row in df.iterrows():
#         question_len.append(len(row["Question"]))


# fig = plt.figure(figsize =(10, 7))
 
# # Creating plot
# plt.boxplot(question_len)
 
# # show plot
# plt.show()

df = df.reset_index()

In [8]:
def clean_sentence(s, stopWords=False):
    s = s.lower().strip()
    #s = re.sub(r'[^a-z0-9\s]',' ',s)
    if stopWords:
        s = remove_stopwords(s)
    return s

def get_cleaned(df, stopWords=False):
    sents = df[["Question"]]
    cleaned_sentences = []
    for index, row in df.iterrows():
        cleaned = clean_sentence(row["Question"], stopWords)
        cleaned_sentences.append(cleaned)
    return cleaned_sentences
cleaned_s_with_not_stopword = get_cleaned(df, stopWords=True)

In [14]:
from bert_serving.client import BertClient
bc = BertClient()

In [15]:
#run this cell only once !!!!!!!!!!!!!!!!!!!!!!
questions_embedding = []
for question_ in cleaned_s_with_not_stopword:
    questions_embedding.append(bc.encode([question_]))

C:\Users\Maha\anaconda3\envs\bert\lib\site-packages\bert_serving\client\__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [47]:
def retrieveAndPrintAnswer(question_embedding, sentence_embeddings, df, sentences):
    max_sim = -1
    index_sim = -1
    potential_questions_indexes = []
    potential_questions = []
    answers = {}
    for index, faq_embedding in enumerate(sentence_embeddings):
        sim = cosine_similarity(faq_embedding, question_embedding)[0][0]    
        if sim>max_sim:
            max_sim = sim
            index_sim = index
            potential_questions_indexes.append(index_sim)
            potential_questions.append(df.iloc[index_sim,1])
    if(len(potential_questions)>11):
        potential_questions_indexes = potential_questions_indexes[1:11]
        potential_questions= potential_questions[1:11]
    potential_questions_indexes.reverse()
    potential_questions.reverse()
    answers = dict( zip(potential_questions_indexes,potential_questions))
    return answers 
#     print("\n")
#     print("Question:", question)
#     print("\n")
#     print("Question retrieved:", df.iloc[index_sim,1])
#     print("\n")
#     print("Answer retrieved:",df.iloc[index_sim,2])

In [48]:
def getAnswers(question):
    question = clean_sentence(question, stopWords=True)
    question_embedding = bc.encode([question])
    answers = retrieveAndPrintAnswer(question_embedding, questions_embedding, df, cleaned_s_with_not_stopword)
    return answers   

In [49]:
#test
answers = getAnswers("Add Variable Editor to Custom Table")

In [50]:
import json
print(json.dumps(answers, sort_keys=False, indent=4))

{
    "484": "We are on Kingston version.I want to hide the new and Edit button from Problem form under the incidentrelated listI configured the list control with the below script.The EDIT button is invisible but not the NEW button.I want to add some condition. If the problem ticket is inactive then hide the buttons.The problem is it is not working evenwithoutcondition.",
    "172": "Is it possible to dynamically change column labels in a List view using JavaScript?",
    "94": "I set field in table is password(1 way encripted) but after create record,admin can't view this password.How can I make admin view this ? pls help me!",
    "66": "We have a custom table to manage our applications, however one of the applications have different sys_id's in our prod and dev instances. This likely occurred because we created the entry in prod, rather than copy it over. I was wondering if there was a way to change the sys_id?Thanks!",
    "62": "I am developing a scoped application for the Service